# Lab 3 - Reinforcement Learning
## Cliff Walking
Marta Simón, Ana Rotella, Blanca Gómez, Enrique Gil and María José Medina

## Description

The board is a 4x12 matrix, with (using NumPy matrix indexing):

- [3, 0] as the start at bottom-left

- [3, 11] as the goal at bottom-right

- [3, 1..10] as the cliff at bottom-center

If the agent steps on the cliff, it returns to the start. An episode terminates when the agent reaches the goal.

### Actions

There are 4 discrete deterministic actions:

- 0: move up

- 1: move right

- 2: move down

- 3: move left

### Observations

There are 3x12 + 1 possible states. In fact, the agent cannot be at the cliff, nor at the goal (as this results in the end of the episode). It remains all the positions of the first 3 rows plus the bottom-left cell. The observation is simply the current position encoded as flattened index.

### Recompensa

Each time step incurs -1 reward, and stepping into the cliff incurs -100 reward.

## Implementation

In [1]:
# Common imports
import numpy as np
import os
import gym
import matplotlib
import random
import itertools
import sys
from collections import defaultdict
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
import plotting

matplotlib.style.use('ggplot')
%matplotlib inline

C:\Users\Maria\anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Maria\anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Maria\anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [4]:
##########  Using gym
import gym

First, we create the Cliff Walking environment

In [5]:
# selecting the gym environment
env = gym.make('CliffWalking-v0')

In [6]:
# Initialization of the environment with reset
env.seed(42)
obs = env.reset()

Then, we check that the number of states and actions coincides with the environment description

In [7]:
# 4x12 grid = 48 states
print ("Number of states:", env.nS)
# either go left, up, down or right
print ("Number of actions that an agent can take:", env.nA)

Number of states: 48
Number of actions that an agent can take: 4


We then check in which position of the board the agent is. 

In [8]:
obs

36

Since the position is represented using a flatmap, obs = 36 means that the agent is at position 36 in the board, that is, the start.
We then plot the environment to check that the assumption is correct.

In [9]:
# An environment can be visualized by calling its render()
img = env.render()
img

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



The x represents the agent and the T the goal postion, Therefore, the agent is in fact at the start.
One can also check where the agent is and what are its options with the following code:

In [10]:
# Where am I? -> in "x" state
print ("Current state", env.s)
# What are my options? -> 4 action
print ("Transitions from current state:", env.P[env.s])

Current state 36
Transitions from current state: {0: [(1.0, 24, -1, False)], 1: [(1.0, 36, -100, False)], 2: [(1.0, 36, -1, False)], 3: [(1.0, 36, -1, False)]}


The dictionary above represents:

    action : (prob, obs, reward, done)
- action : possible action that the agent can take
- obs : observation were the agent to take the action
- reward : reward were the agent to take the action
- done : True if the agent would reach its goal in the next step

Now we will make the agent take an action (move up) to see how the environment changes

In [11]:
action = 0  # move up
obs, reward, done, info = env.step(action)
print(f"Observation: {obs}")
print(f"Reward: {reward}")
print(f"Done: {done}")
print(f"Info: {info}")

Observation: 24
Reward: -1
Done: False
Info: {'prob': 1.0}


In [12]:
env.render()

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T



The x has moved up one position. However, if the agent were to move right in stead of left, the x would not move as the positions in the cliff are not feasible.

In [13]:
env.reset()
action = 1  # move up
obs, reward, done, info = env.step(action)
print(f"Observation: {obs}")
print(f"Reward: {reward}")
print(f"Done: {done}")
print(f"Info: {info}")
env.render()

Observation: 36
Reward: -100
Done: False
Info: {'prob': 1.0}
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



### Basic Policy

Walk to the goal without falling into the cliff

In [14]:
env.seed(42)

def basic_policy(obs):
    action = 1 # By default go right
    if obs >= 36: # If the agent is in the lower row, go up
        action = 0
    elif (obs == 11) or ( obs == 23) or ( obs == 35): # If the agent is in the last column, go down
        action = 2
    return action

totals = []
# for episode in range(500):
for episode in range(1):
    episode_rewards = 0
    obs = env.reset()
    # for step in range(200):
    for step in range(13):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        env.render()
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  x  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  x  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o

In [15]:
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

# Well, as expected, this strategy is a bit too basic: the best it did was to keep the pole up for only 68 steps. 
# This environment is considered solved when the agent keeps the pole up for 200 steps.

(-13.0, 0.0, -13, -13)

In [16]:
env.reset()

36

## Sarsa (On-policy)

In [17]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        pi = np.ones(nA, dtype=float) * (epsilon/nA)
        best_action = np.argmax(Q[observation])
        pi[best_action] += (1.0 - epsilon)
        return pi
    
    return policy_fn

In [18]:
%%time 

next_state = obs

# generate a episode in cliffwalking environment for above sample_policy
def generate_episode(policy, verbose=False):
    episode = []
    env = CliffWalkingEnv()
    curr_state = env.reset()
    probs = policy(curr_state)
    action = np.random.choice(np.arange(len(probs)), p=probs)
    
    while True:
        if verbose:
            print ("Current observation:")
            print ("Current poistion:", curr_state)
            #print (env.render())
        
        next_obs, reward, is_terminal, _ = env.step(action)
        
        if verbose:
            print ("Action taken:", actions[action])
            print ("Next observation:", next_obs)
            print ("Reward recieved:", reward)
            print ("Terminal state:", is_terminal)
            #print (env.render())
            print ("-"*20)
        episode.append((curr_state, action, reward))
        
        # Pick the next action
        next_probs = policy(next_state)
        next_action = np.random.choice(np.arange(len(next_probs)), p=next_probs)
    
        curr_state = next_obs
        action = next_action

        if (is_terminal):
            break

    return episode
    
Q = defaultdict(lambda: np.zeros(env.action_space.n))
policy = make_epsilon_greedy_policy(Q, 0.1, env.action_space.n)
e = generate_episode(policy)
#print ("Episode:", e)
print ("Length of episode:", len(e))

Length of episode: 291847
CPU times: total: 7.69 s
Wall time: 7.85 s


In [19]:
def sarsa(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """
    SARSA algorithm: On-policy TD control. Finds the optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
    
    Returns:
        A tuple (Q, stats).
        Q is the optimal action-value function, a dictionary mapping state -> action values.
        stats is an EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))

    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    for i_episode in range(num_episodes):
        # Print out which episode we're on, useful for debugging.
        if (i_episode + 1) % 100 == 0:
            print("\rEpisode {}/{}.".format(i_episode + 1, num_episodes), end="")
            sys.stdout.flush()
    
        # Reset the environment and pick the first action
        state = env.reset()
        probs = policy(state)
        action = np.random.choice(np.arange(len(probs)), p=probs)
        
        # Approach : 1
        # Takes a lot a lot lot of time to run
        # ep = generate_episode(policy)
        # for i in range(len(ep)-1):
        #     state, action, reward = ep[i]
        #     next_state, next_action, next_reward = ep[i+1]
        
        #     td_target = reward + discount_factor * Q[next_state][next_action]
        #     td_error = td_target - Q[state][action]
        #     Q[state][action] += alpha * td_error

        #     stats.episode_rewards[i_episode] += reward
        #     stats.episode_lengths[i_episode] = t

        # Approach : 2
        # One step in the environment
        for t in itertools.count():
            # Take a step
            next_state, reward, is_terminal, _ = env.step(action)
            
            # Pick the next action
            next_probs = policy(next_state)
            next_action = np.random.choice(np.arange(len(next_probs)), p=next_probs)
            
            td_target = reward + discount_factor * Q[next_state][next_action]
            td_error = td_target - Q[state][action]
            Q[state][action] += alpha * td_error

            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t

            if is_terminal:
                break
            
            state = next_state
            action = next_action

    return Q, stats

In [20]:
%%time
Q, stats = sarsa(env, 200)

AttributeError: module 'plotting' has no attribute 'EpisodeStats'